# Lademanagementsimulation   


### Simulations Parameter

In [1]:
solarpeakleistung = 125 #Einheit Kilowattpeak (kWP)
anzahl_bevs_pro_tag = 50
ladeleistung_pro_bev = 11 #Einheit kW
maximale_ladezeit = 30 #Einheit Minuten
minute_interval = 15 #Einheit Minuten

### Simulations Eingangsdaten

In [2]:
import simulationInputDataVisualisation
simulationInputDataVisualisation.create_jupyter_dash_app(solarpeakleistung)

### Verteilungsalgorithmus

In [3]:
import distributionAlgorithmPolling
import simulationClasses
import simulationData
import simulationOutputDataVisualisation
import distributionAlgorithmForecast
import distributionAlgorithmForecastPolling

simulation_day = simulationClasses.SimulationDay(anzahl_bevs_pro_tag)
bev_data = simulationData.BevData()
table_dict = simulationData.TableDict()
simulation_data = simulationData.SimulationData()

# Auswahl des Algorithmus
# algorithm = distributionAlgorithmPolling.Algorithm.POLLING_FIFO
# algorithm = distributionAlgorithmPolling.Algorithm.POLLING_EVEN_DISTRIBUTION
# distributionAlgorithmPolling.start_simulation(solarpeakleistung, maximale_ladezeit, ladeleistung_pro_bev, simulation_day, bev_data, table_dict, simulation_data, algorithm, minute_interval)

#distributionAlgorithmForecast.start_algorithm(simulation_data, simulation_day, maximale_ladezeit, solarpeakleistung, 
  #                  bev_data, table_dict, ladeleistung_pro_bev, minute_interval)

distributionAlgorithmForecastPolling.start_simulation(solarpeakleistung, ladeleistung_pro_bev,
                     simulation_day, bev_data, table_dict, simulation_data, minute_interval)



Minute:  480
Charging BEVs after updating parking end:  []
Waiting BEVs:  [2, 7, 10, 11, 13, 17, 23, 26, 36, 40, 42, 43, 48]
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  1
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  1
NUMBER OF FREE CHARGING STATIONS:  1
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  1
Charging BEV Liste vor Residual time Berechnung:  [2]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 2:  8.01
Restladezeit:  775.1612903225807
Charging BEVs:  [2]


Minute:  495
Charging BEVs after updating parking end:  [2]
Waiting BEVs:  [7, 10, 11, 13, 17, 23, 26, 36, 40, 42, 43, 48]
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  1
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  1
Charging BEV Liste vor Residual time Berechnung:  [2]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 2:  7.8549999999999995
Restladezeit:  80.28960817717206
Charging BEVs:  [2]

Charging BEV Liste vor Residual time Berechnung:  [10, 11, 17, 23, 26, 36, 40, 42, 43, 48, 0, 1, 3]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 10:  0.9350000000000005
Restladezeit:  15.517021276595752
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 11:  4.665000000000001
Restladezeit:  77.41914893617023
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 17:  3.9416666666666647
Restladezeit:  65.41489361702124
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 23:  4.972499999999995
Restladezeit:  82.52234042553182
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 26:  2.0941666666666747
Restladezeit:  34.754255319149074
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 36:  4.6225000000000005
Restladezeit:  76.71382978723405
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 40:  4.36
Restladezeit:  72.35744680851064
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 42:  1.1725000000000005
Restladezeit:  19.458510638

Charging BEV Liste vor Residual time Berechnung:  [10, 11, 17, 23, 26, 36, 40, 42, 43, 48, 0, 1, 3]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 10:  0.03115384615384631
Restladezeit:  0.4775943396226439
Die Restladezeit des BEVS 10 endet vor dem nächsten Interval
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  4
Ladestart für ausgewähltes BEV:  600.4775943396227
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 11:  4.122692307692309
Restladezeit:  63.20165094339624
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 17:  3.0378205128205105
Restladezeit:  46.57036163522009
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 23:  4.10225838264299
Restladezeit:  62.888395016932634
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 26:  1.6175230111768688
Restladezeit:  24.796932954362376
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 36:  3.7186538461538468
Restladezeit:  57.00766509433963
Restladeenergie bis zur fairen Ladeen

Ausgewähltes BEV das frei gewordenen Platz belegen darf:  35
Ladestart für ausgewähltes BEV:  600.0
Charging BEV Liste vor Residual time Berechnung:  [10, 11, 17, 23, 26, 36, 40, 42, 43, 48, 0, 1, 3]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 10:  0.0
Restladezeit:  0.0
Die Restladezeit des BEVS 10 endet vor dem nächsten Interval
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  38
Ladestart für ausgewähltes BEV:  600.0
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 11:  4.122692307692309
Restladezeit:  63.20165094339624
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 17:  3.0378205128205105
Restladezeit:  46.57036163522009
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 23:  4.10225838264299
Restladezeit:  62.888395016932634
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 26:  1.6175230111768688
Restladezeit:  24.796932954362376
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 36:  3.7186538461538468
Restladezeit

Restladezeit:  16.152712264150946
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 0:  4.406153846153847
Restladezeit:  67.54716981132077
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 1:  5.496153846153845
Restladezeit:  84.25707547169809
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 3:  0.0
Restladezeit:  0.0
Die Restladezeit des BEVS 3 endet vor dem nächsten Interval
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  None
Charging BEV Liste vor Residual time Berechnung:  [10, 11, 17, 23, 26, 36, 40, 42, 43, 48, 0, 1, 3]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 10:  0.0
Restladezeit:  0.0
Die Restladezeit des BEVS 10 endet vor dem nächsten Interval
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  None
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 11:  4.122692307692309
Restladezeit:  63.20165094339624
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 17:  3.0378205128205105
Restladezeit:  46.57036163

...

Restladezeit:  -38.521375964358384
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 26:  -82.65266117053235
Restladezeit:  -2841.0831773355653
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 36:  -1.8610813207350878
Restladezeit:  -63.972372542059524
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 40:  -2.0166089176745636
Restladezeit:  -69.3184416585093
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 43:  -0.8608671729950368
Restladezeit:  -29.591246167747215
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 0:  -1.1153003593918074
Restladezeit:  -38.3370728040645
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 1:  0.34445001425987165
Restladezeit:  11.840043951248058
Die Restladezeit des BEVS 1 endet vor dem nächsten Interval
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  None
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 4:  2.611088632017994
Restladezeit:  89.75294784099782
Restladeenergie bis zur faire

Ausgewähltes BEV das frei gewordenen Platz belegen darf:  None
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 22:  0.4735306492128908
Restladezeit:  16.27703140320881
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 24:  -0.6173265304575217
Restladezeit:  -21.219837277678543
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 28:  1.173530649212891
Restladezeit:  40.338667120316686
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 32:  -1.365387932194519
Restladezeit:  -46.93352433857098
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 33:  0.9710306492128913
Restladezeit:  33.377979645010484
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 39:  -3.018940289432201
Restladezeit:  -103.77234499431114
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 41:  1.356030649212891
Restladezeit:  46.611879289419804
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 34:  1.5860306492128906
Restladezeit:  54.517845310755234
Restladee

Ausgewähltes BEV das frei gewordenen Platz belegen darf:  None
Charging BEV Liste vor Residual time Berechnung:  [11, 17, 23, 26, 36, 40, 43, 0, 1, 4, 8, 14, 16, 18, 19, 22, 24, 28, 32, 33, 39, 41, 34, 35, 38, 44, 9, 15, 45]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 11:  -3.179100027239758
Restladezeit:  -109.27763823384393
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 17:  -5.0681714722961555
Restladezeit:  -174.21213674032617
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 23:  -3.8203613004195622
Restladezeit:  -131.32020274061713
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 26:  -159.15864772881633
Restladezeit:  -5470.8820041118215
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 36:  -4.560780522697281
Restladezeit:  -156.77119931831825
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 40:  -4.716308119636757
Restladezeit:  -162.11726843476802
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 43:  -3.5605

Charging BEVs:  []


Minute:  720
Charging BEVs after updating parking end:  []
Waiting BEVs:  [37]
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
NUMBER OF FREE CHARGING STATIONS:  5
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
Charging BEV Liste vor Residual time Berechnung:  [37]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 37:  4.115
Restladezeit:  4.6921322690992024
Die Restladezeit des BEVS 37 endet vor dem nächsten Interval
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  None
Charging BEVs:  [37]


Minute:  735
Charging BEVs after updating parking end:  []
Waiting BEVs:  []
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
NUMBER OF FREE CHARGING STATIONS:  5
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
Charging BEVs:  []


Min

Charging BEV Liste vor Residual time Berechnung:  [30, 31, 46, 47]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 30:  0.6175000000000028
Restladezeit:  3.7283018867924698
Die Restladezeit des BEVS 30 endet vor dem nächsten Interval
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  21
Ladestart für ausgewähltes BEV:  813.7283018867925
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 31:  0.6175000000000028
Restladezeit:  3.7283018867924698
Die Restladezeit des BEVS 31 endet vor dem nächsten Interval
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  49
Ladestart für ausgewähltes BEV:  813.7283018867925
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 46:  0.6175000000000028
Restladezeit:  3.7283018867924698
Die Restladezeit des BEVS 46 endet vor dem nächsten Interval
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  None
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 47:  0.463125000000002
Restladezeit:  2.7962264150943517


### Ergebnis Algorithmus

In [4]:
simulationOutputDataVisualisation.print_algorithm_result(bev_data, simulation_data, anzahl_bevs_pro_tag)

>**Ergebnis**<br>Geladene BEVs: 50 von 50<br>Aufgeladene Solarenergie insgesamt: 527.71 kWh<br>Ungenutzte Solarenergie insgesamt: 229.095 kWh

### Tabellarische Übersicht Park- und Ladeverlauf der BEVs

In [5]:
simulationOutputDataVisualisation.create_tabular_overview_per_minute_slider(table_dict, minute_interval)

interactive(children=(IntSlider(value=480, description='Minute: ', max=960, min=480, step=15), Output()), _dom…

In [6]:
import figureGeneration

figureGeneration.create_charging_power_figure(simulation_day, solarpeakleistung, bev_data, minute_interval)

{2: {480: -2.0, 495: 3.25, 510: 4.25, 525: 6.875, 540: 6.333333333333333, 555: 10.333333333333334, 561.0: 10.333333333333334}, 7: {510: 4.25, 525: 6.875, 540: 6.333333333333333, 555: 9.0, 570: 7.8, 570.5576923076923: 7.8}, 10: {540: 6.333333333333333, 555: 9.0, 570: 7.0, 585: 3.3076923076923075, 600: 3.913846153846154, 600.4775943396227: 3.913846153846154}, 11: {561.0: 10.333333333333334, 570: 7.0, 585: 3.3076923076923075, 600: 3.923076923076923, 615: 1.6916666666666667, 630: 1.7413793103448276, 645: 1.7327586206896552, 660: 1.7241379310344827, 672.2734613718154: 1.7455172413793103}, 13: {555.0: 10.333333333333334, 570: 7.8, 577.0897435897436: 7.8}, 17: {555.0: 10.333333333333334, 570: 7.0, 585: 3.3076923076923075, 600: 3.923076923076923, 615: 1.6916666666666667, 630: 1.7413793103448276, 645: 1.7327586206896552, 660: 1.7241379310344827, 668.7511808611865: 1.7455172413793103}, 23: {570.5576923076923: 7.8, 585: 3.3076923076923075, 600: 3.923076923076923, 615: 1.6916666666666667, 630: 1.7

In [7]:
figureGeneration.create_bev_number_figure(simulation_data)

In [8]:
import postOptimization

postOptimization.sort_waiting_list_descending_by_parking_end(simulation_day)

OrderedDict([(12, 1248), (20, 1218), (6, 1170), (27, 1152), (49, 1140), (30, 1110), (31, 1074), (5, 1062), (46, 1044), (29, 1038), (9, 1032), (47, 1008), (37, 1008), (34, 996), (21, 990), (45, 984), (35, 978), (25, 966), (1, 960), (17, 948), (7, 912), (41, 906), (2, 906), (28, 888), (18, 888), (4, 888), (33, 876), (16, 876), (11, 870), (0, 870), (43, 864), (10, 858), (22, 852), (14, 846), (8, 846), (23, 834), (19, 834), (36, 822), (40, 804), (24, 804), (15, 792), (44, 762), (32, 755), (38, 732), (48, 702), (39, 702), (3, 696), (13, 690), (42, 672), (26, 666)])
